<a href="https://colab.research.google.com/github/xanasa14/MLImplementations/blob/master/NetflixVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import re
nf=pd.read_csv("../input/netflix-shows/netflix_titles.csv")
nf.head()